# Profitable Apps Profiles for the App Store and Google Play Markets

The aim of this project is to find profiles for profitable mobile apps in App Store and Google Play markets. This can help developers to understand what types of apps they should build to generate profits. Given that we only develop free apps, so the main source of revenue is in-app ads. The revenue of an app will largely depending on how many customers it has. Our goal of this project is to identify what kinds of apps are likely to attract more users.

In [1]:
from csv import reader

#The Google Play data set#
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

#The App Store data set#
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

In [2]:
def explore_data(dataset, start, end, rows_and_columns = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') #adds a new (empty) line after each row
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

print(android_header)
print('\n')
explore_data(android, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite \xe2\x80\x93 FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


('Number of rows:', 10841)
('Number of columns:', 13)


In [3]:
print(ios_header)
print('\n')
explore_data(ios, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


('Number of rows:', 7197)
('Number of columns:', 16)


In [4]:
print(android[10472])
print('\n')
print(android_header)
print('\n')
print(android[0])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Delete wrong data

In [5]:
del android[10472]

Identify duplicate apps

In [6]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Number of duplicate apps: ', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps: ', duplicate_apps[:15])

('Number of duplicate apps: ', 1181)


('Examples of duplicate apps: ', ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software'])


We need to remove duplicate rows and keep only one entry per app. When examine the duplicate rows, we found the main defference is the number of reviews. The different numbers show that the data was collected at different times. We will keep the row has the highest number of reviews because the higher the number of reviews, the more reliable the ratings. 
To do that, we will:
1. Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
2. Use the dictionary to create a new data set, which will have only one entry per app. 

In [7]:
reviews_max ={}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews 

In [8]:
print('Expected length', len(android) - 1181)
print('Actual length', len(reviews_max))

('Expected length', 9659)
('Actual length', 9659)


We will use reviews_max dictionary to remove duplicates. In the code below: 
1. We isolate the name of the app and the numbers of reviews. 
2. We add the current row(app) to the android_clean list, and the app nmae(name) to the already_clean list if:

1)The number of reviews of the current app matches the number of reviews of that app as described in reviews_max dictionary, and 

2)The name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.
    

In [9]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)

Now we need to confirm that the number of rows is 9,659. 

In [10]:
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite \xe2\x80\x93 FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


('Number of rows:', 9659)
('Number of columns:', 13)


# Remove Non-English Apps

All characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it. If an app name contains a character that is greater than 127, then it probably means that the app has a non-English name. 

In [11]:
def is_english(string):
    
    for character in string:
        if ord(character) > 127:
            return False
    
    return True

However, some English app names use emojis or other symbols(™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. Because of this, we'll remove useful apps if we use the function in its current form. 

To minimize the impact of data loss, we'll only remove an app if its name has more than three non-ASCII characters:

In [12]:
def is_english(string):
    non_ascii = 0.0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

In [13]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
for app in ios:
    name = app[1]
    if is_english(name):
        ios_english.append(app)
        
explore_data(android_english, 0, 3, True)
print('\n')
explore_data(ios_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite \xe2\x80\x93 FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


('Number of rows:', 9500)
('Number of columns:', 13)


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.

# Isolate Free Apps

In [14]:
android_final = []
ios_final = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_final.append(app)

for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)

print(len(android_final))
print(len(ios_final))

8760
3169


# Most Common Apps by Genre

# Part One

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

# Part Two

We will build two functions we can use to analyze the frequency tables: 

    1.One function to generate frequency tables that show percentages
    2.Another function that we can use to display the perentages in a decending order

In [15]:
def freq_table(dataset, index):
    table = {}
    total = 0.0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages


def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print entry[1], ':', entry[0]

# Part Three

We start by examining the frequency table for the prime_genre column of the App Store data set.

In [16]:
display_table(ios_final, -5)

Games : 58.5358157147
Entertainment : 7.82581255917
Photo & Video : 5.04891132849
Education : 3.72357210476
Social Networking : 3.28179236352
Shopping : 2.52445566425
Utilities : 2.39823288104
Sports : 2.17734301041
Music : 2.0511202272
Health & Fitness : 1.98800883559
Productivity : 1.70400757337
Lifestyle : 1.54622909435
News : 1.32533922373
Travel : 1.13600504891
Finance : 1.10444935311
Weather : 0.852003786683
Food & Drink : 0.82044809088
Reference : 0.536446828653
Business : 0.536446828653
Book : 0.378668349637
Navigation : 0.189334174819
Medical : 0.189334174819
Catalogs : 0.126222783212


We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

In [17]:
display_table(android_final, 1) # Category

FAMILY : 18.9383561644
GAME : 9.65753424658
TOOLS : 8.48173515982
BUSINESS : 4.64611872146
PRODUCTIVITY : 3.93835616438
LIFESTYLE : 3.91552511416
FINANCE : 3.72146118721
MEDICAL : 3.5502283105
SPORTS : 3.33333333333
PERSONALIZATION : 3.28767123288
COMMUNICATION : 3.25342465753
HEALTH_AND_FITNESS : 3.09360730594
PHOTOGRAPHY : 2.97945205479
NEWS_AND_MAGAZINES : 2.80821917808
SOCIAL : 2.64840182648
TRAVEL_AND_LOCAL : 2.3401826484
SHOPPING : 2.24885844749
BOOKS_AND_REFERENCE : 2.14611872146
DATING : 1.86073059361
VIDEO_PLAYERS : 1.80365296804
MAPS_AND_NAVIGATION : 1.38127853881
FOOD_AND_DRINK : 1.23287671233
EDUCATION : 1.17579908676
ENTERTAINMENT : 0.958904109589
AUTO_AND_VEHICLES : 0.924657534247
LIBRARIES_AND_DEMO : 0.901826484018
WEATHER : 0.787671232877
HOUSE_AND_HOME : 0.787671232877
EVENTS : 0.719178082192
ART_AND_DESIGN : 0.650684931507
PARENTING : 0.639269406393
BEAUTY : 0.60502283105
COMICS : 0.582191780822


The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:

In [18]:
display_table(android_final, -4)

Tools : 8.4703196347
Entertainment : 6.08447488584
Education : 5.38812785388
Business : 4.64611872146
Productivity : 3.93835616438
Lifestyle : 3.90410958904
Finance : 3.72146118721
Medical : 3.5502283105
Sports : 3.40182648402
Personalization : 3.28767123288
Communication : 3.25342465753
Action : 3.10502283105
Health & Fitness : 3.09360730594
Photography : 2.97945205479
News & Magazines : 2.80821917808
Social : 2.64840182648
Travel & Local : 2.32876712329
Shopping : 2.24885844749
Books & Reference : 2.14611872146
Simulation : 2.05479452055
Dating : 1.86073059361
Arcade : 1.82648401826
Video Players & Editors : 1.78082191781
Casual : 1.73515981735
Maps & Navigation : 1.38127853881
Food & Drink : 1.23287671233
Puzzle : 1.14155251142
Racing : 1.00456621005
Role Playing : 0.947488584475
Strategy : 0.924657534247
Auto & Vehicles : 0.924657534247
Libraries & Demo : 0.901826484018
Weather : 0.787671232877
House & Home : 0.787671232877
Events : 0.719178082192
Adventure : 0.662100456621
Beauty 

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

# Most Popular Apps by Genre on the App Store

One way to find out the most popular apps is to calculate the average numbers of installs. For Google Play data set, we can use Install column to do that. But in App Store data set, this information is missing. We will use total numbers of ratings instead. 

The numbers of user ratings per app genre on the App Store are calculated below:

In [19]:
import operator
genre_ios = freq_table(ios_final, -5)

unsorted_rating_dic = {}

for genre in genre_ios:
    total = 0.0
    len_genre = 0.0
    
    for app in ios_final:
        genre_app = app[-5]
        if genre_app == genre:
            total += float(app[5])
            len_genre += 1
    average_tot_rating = total / len_genre
    unsorted_rating_dic[genre] = average_tot_rating

sorted_rating_dic = sorted(unsorted_rating_dic.items(), key = operator.itemgetter(1), reverse = True)
for item in sorted_rating_dic:
    print (item) 

('Navigation', 86090.33333333333)
('Reference', 79350.4705882353)
('Social Networking', 72916.54807692308)
('Music', 58205.03076923077)
('Weather', 54215.2962962963)
('Book', 46384.916666666664)
('Food & Drink', 33333.92307692308)
('Finance', 32367.02857142857)
('Travel', 31358.5)
('Photo & Video', 28441.54375)
('Shopping', 27816.2)
('Health & Fitness', 24037.634920634922)
('Sports', 23008.898550724636)
('Games', 22985.211320754715)
('Productivity', 21799.14814814815)
('News', 21750.071428571428)
('Utilities', 19900.473684210527)
('Lifestyle', 16739.34693877551)
('Entertainment', 14364.774193548386)
('Business', 7491.117647058823)
('Education', 7003.983050847458)
('Catalogs', 4004.0)
('Medical', 612.0)


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [20]:
for app in ios_final:
    if app[-5] == 'Navigation':
        print app[1], ':', app[5]

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [21]:
for app in ios_final:
    if app[-5] == 'Reference':
        print app[1], ':', app[5]

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
Jishokun-Japanese English Dictionary & Translator : 0


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

1. Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.
2. Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.
3. Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

# Most Popular Apps by Genre on Google Play

In [22]:
category_android = freq_table(android_final, 1)

unsorted_install_dic = {}
for category in category_android:
    total = 0.0
    len_category = 0.0 
    
    for app in android_final:
        category_app = app[1]
        if category_app == category:
            install = app[5]
            install = install.replace(',', '')
            install = install.replace('+', '')
            total += float(install)
            len_category += 1
    average_install = total / len_category
    unsorted_install_dic[category] = average_install

sorted_intall_dic = sorted(unsorted_install_dic.items(), key = operator.itemgetter(1), reverse = True)
for item in sorted_intall_dic:
    print (item)

('COMMUNICATION', 38550548.03859649)
('VIDEO_PLAYERS', 24878048.860759493)
('SOCIAL', 23628689.23275862)
('PHOTOGRAPHY', 17840110.40229885)
('PRODUCTIVITY', 16787331.344927534)
('GAME', 15571586.690307328)
('TRAVEL_AND_LOCAL', 14120454.07804878)
('ENTERTAINMENT', 11767380.952380951)
('TOOLS', 10902378.834454913)
('NEWS_AND_MAGAZINES', 9626407.357723577)
('BOOKS_AND_REFERENCE', 8329168.936170213)
('SHOPPING', 7103190.78680203)
('PERSONALIZATION', 5240358.986111111)
('WEATHER', 5212877.101449275)
('HEALTH_AND_FITNESS', 4219697.055350553)
('MAPS_AND_NAVIGATION', 4115374.214876033)
('SPORTS', 3750580.6438356163)
('FAMILY', 3716053.755274262)
('ART_AND_DESIGN', 1986335.0877192982)
('FOOD_AND_DRINK', 1951283.8055555555)
('EDUCATION', 1833495.145631068)
('BUSINESS', 1712290.1474201474)
('LIFESTYLE', 1447458.976676385)
('HOUSE_AND_HOME', 1385541.463768116)
('FINANCE', 1365500.4049079753)
('DATING', 861409.5521472392)
('COMICS', 859042.1568627451)
('AUTO_AND_VEHICLES', 654074.8271604938)
('LIBR

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [23]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print app[0], ':', app[5]

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:

In [24]:
under_100_m = []

for app in android_final:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3437620.895348837

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.). 

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play. 

In [25]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print app[0], ':', app[5]

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

In [26]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print app[0], ':', app[5] 

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Audiobooks from Audible : 100,000,000+


There are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [27]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print app[0], ':', app[5]

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

# Conclusions

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.